In [1]:
import sys
import os

# Determine the absolute path to the root directory (assuming the script is one level down)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to sys.path if it’s not already there
if root_path not in sys.path:
    sys.path.insert(0, root_path)


In [5]:
import sqlite3

def list_tables(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Execute a query to get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Close the connection
    conn.close()

    # Return a list of table names (each item in 'tables' is a tuple)
    return [table[0] for table in tables]

In [6]:
list_tables('../data/trading_system.db')

['daily_prices',
 'company_info',
 'balance_sheet',
 'income_statement',
 'cash_flow']

In [4]:
from src.strategies.momentum.awesome_oscillator_strat import AwesomeOscillatorStrategy

In [2]:
from src.database.config import DatabaseConfig
from src.database.engine import create_db_engine
db_config = DatabaseConfig.default()

In [5]:
# Instantiate the strategy, passing the db_config
ao_strategy = AwesomeOscillatorStrategy(db_config, params={'short_period': 5, 'long_period': 34})

# Now you can use ao_strategy to generate signals, and it will use the database engine
signals_df = ao_strategy.generate_signals(ticker='RELIANCE.BO')

In [6]:
signals_df

,close,high,low,signal,position,return,cumulative_return,exit_type
date,,,,,,,,
2024-12-23,1222.099976,1227.000000,1213.150024,0,0.0,0.000000,0.000000,none
2024-12-24,1223.500000,1233.449951,1221.449951,0,0.0,0.000000,0.000000,none
2024-12-26,1216.599976,1228.000000,1214.449951,0,0.0,0.000000,0.000000,none
2024-12-27,1220.949951,1227.599976,1216.650024,0,0.0,0.000000,0.000000,none
2024-12-30,1210.900024,1223.349976,1208.650024,0,0.0,0.000000,0.000000,none
2024-12-31,1215.449951,1219.050049,1206.400024,0,0.0,0.000000,0.000000,none
2025-01-01,1221.650024,1226.199951,1212.099976,0,0.0,0.000000,0.000000,none
2025-01-02,1241.650024,1244.500000,1220.250000,0,0.0,0.000000,0.000000,none
2025-01-03,1251.349976,1262.300049,1235.599976,0,0.0,0.000000,0.000000,none


In [3]:
import pandas as pd

tickers = pd.read_excel("../data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()

In [8]:
# Now you can use ao_strategy to generate signals, and it will use the database engine
signals_df = ao_strategy.generate_signals(ticker=all_tickers[200])
signals_df

,close,high,low,signal,position,return,cumulative_return,exit_type
date,,,,,,,,
2024-12-23,4353.649902,4399.000000,4287.250000,0,0.0,0.000000,0.000000,none
2024-12-24,4344.649902,4441.799805,4300.000000,0,0.0,0.000000,0.000000,none
2024-12-26,4357.899902,4367.200195,4315.000000,0,0.0,0.000000,0.000000,none
2024-12-27,4396.399902,4445.350098,4355.250000,0,0.0,0.000000,0.000000,none
2024-12-30,4129.000000,4320.000000,4104.299805,0,0.0,0.000000,0.000000,none
2024-12-31,4135.000000,4226.250000,4007.050049,0,0.0,0.000000,0.000000,none
2025-01-01,4239.399902,4258.299805,4160.000000,0,0.0,0.000000,0.000000,none
2025-01-02,4233.250000,4247.500000,4154.649902,0,0.0,0.000000,0.000000,none
2025-01-03,4121.700195,4249.299805,4102.549805,0,0.0,0.000000,0.000000,none


In [22]:
from src.database.utils import query_to_dataframe


In [31]:
ticker = all_tickers[500]
a = query_to_dataframe(f"SELECT * FROM daily_prices WHERE ticker = '{ticker}'", db_path='../data/trading_system.db')


In [32]:
a

,index,date,open,high,low,close,volume,Dividends,Stock Splits,ticker,updated_at,data_source
0,0,2019-04-16 00:00:00.000000,611.682461,645.310490,608.783492,632.748291,22985293,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1,1,2019-04-18 00:00:00.000000,634.777646,642.508230,593.322362,621.877197,5415019,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
2,2,2019-04-22 00:00:00.000000,621.345810,624.051503,604.193573,606.947571,1059760,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
3,3,2019-04-23 00:00:00.000000,606.850875,626.177336,606.850875,612.745422,1208860,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
4,4,2019-04-24 00:00:00.000000,615.064622,618.446752,607.092457,616.030945,1016453,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
...,...,...,...,...,...,...,...,...,...,...,...,...
1433,1433,2025-02-03 00:00:00.000000,5834.450195,5919.799805,5745.049805,5857.750000,405813,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1434,1434,2025-02-04 00:00:00.000000,5866.700195,5987.000000,5860.049805,5963.899902,326464,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1435,1435,2025-02-05 00:00:00.000000,6039.000000,6092.250000,5964.549805,6037.299805,341372,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance
1436,1436,2025-02-06 00:00:00.000000,6058.399902,6062.600098,5850.000000,5859.000000,191057,0.0,0.0,POLYCAB.NS,2025-02-09 11:09:58.604788,yfinance


In [30]:
a['date'].duplicated().sum()

np.int64(4)

In [40]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    a = ao_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

Insufficient data for analysis
Insufficient data for AO calculation for KAMAHOLD.BO. Required 36 records.


In [41]:
df

,close,ao,signal,signal_strength,ticker
2025-02-07 00:00:00.000000,1732.099976,31.116458,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,41.226611,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,45.677204,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,1732.099976,49.157350,0.0,0.0,HDFCBANK.BO
2025-02-07 00:00:00.000000,4029.350098,-8.770191,0.0,0.0,TCS.BO
...,...,...,...,...,...
2025-02-07 00:00:00.000000,1526.400024,-93.070141,0.0,0.0,RAYMOND.NS
2025-02-04 00:00:00.000000,1290.500000,-319.025155,0.0,0.0,CDSL.NS
2025-02-05 00:00:00.000000,1348.849976,-293.114421,0.0,0.0,CDSL.NS
2025-02-06 00:00:00.000000,1328.050049,-266.467363,0.0,0.0,CDSL.NS


In [44]:
df['signal_strength'].describe()

count    2014.000000
mean        0.419074
std         3.617582
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        84.697567
Name: signal_strength, dtype: float64

In [9]:
from src.strategies.momentum.cci_oscillator_strat import CCIStrategy

In [10]:
cci_strategy = CCIStrategy(db_config)

In [18]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    a = cci_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

KeyboardInterrupt: 

In [11]:
# Now you can use ao_strategy to generate signals, and it will use the database engine
signals_df = cci_strategy.generate_signals(ticker=all_tickers[200])
signals_df

,close,high,low,cci,signal,signal_strength,position,return,cumulative_return,exit_type
date,,,,,,,,,,
2024-01-31,3670.010742,3709.832204,3624.993310,NaN,0,NaN,0.0,0.0000,0.000000,none
2024-02-01,3639.382812,3716.027646,3580.175390,NaN,0,NaN,0.0,0.0000,0.000000,none
2024-02-02,3547.099121,3626.541992,3534.558335,NaN,0,NaN,0.0,0.0000,0.000000,none
2024-02-05,3610.653564,3657.369970,3475.550766,NaN,0,NaN,0.0,0.0000,0.000000,none
2024-02-06,3643.679932,3677.905321,3601.360265,NaN,0,NaN,0.0,0.0000,0.000000,none
...,...,...,...,...,...,...,...,...,...,...
2025-02-03,3609.550049,3642.100098,3468.550049,-63.263506,0,NaN,0.0,-0.0519,-0.918829,stop_loss
2025-02-04,3716.600098,3729.500000,3618.899902,9.852439,0,NaN,0.0,-0.0519,-0.923042,stop_loss
2025-02-05,3679.850098,3759.949951,3648.100098,28.632135,0,NaN,0.0,-0.0519,-0.927036,stop_loss


In [14]:
signals_df['signal'].describe()

count    252.000000
mean       0.007937
std        0.252352
min       -1.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: signal, dtype: float64

In [15]:
signals_df['signal_strength'].describe()

count     16.000000
mean       6.422699
std       43.604906
min      -66.735259
25%      -12.759391
50%        5.707503
75%       23.902127
max      126.965033
Name: signal_strength, dtype: float64

In [4]:
from src.strategies.momentum.coppock_curve_strat import CoppockCurveStrategy

In [5]:
coppock_strategy = CoppockCurveStrategy(db_config)

In [8]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    print(i)
    a = coppock_strategy.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


Insufficient data for analysis
/tmp/ipykernel_542178/1703485871.py:9: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat([df, a])


102
103
104
105
106
107
108
109


Insufficient data for analysis
/tmp/ipykernel_542178/1703485871.py:9: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat([df, a])


110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184


...

397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414


Insufficient data for analysis
/tmp/ipykernel_542178/1703485871.py:9: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat([df, a])


415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442


Insufficient data for analysis
/tmp/ipykernel_542178/1703485871.py:9: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat([df, a])


443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507


In [15]:
sum(df.signal ==-1)


66

In [16]:
def generate_repeated_array(input_number):
    """
    Generates a numpy array where the input number is repeated 21 times,
    the previous number is repeated 21 times, and so on, until 1 is repeated 21 times.

    Args:
        input_number (int): The starting number for the array generation.

    Returns:
        numpy.ndarray: A 1D numpy array with the described repeated sequence.
    """

    if not isinstance(input_number, int) or input_number <= 0:
        raise ValueError("Input must be a positive integer.")

    repetitions = 21
    number_sequence = np.arange(input_number, 0, -1) # Efficiently create a sequence from input_number down to 1
    repeated_array = np.repeat(number_sequence, repetitions) # Efficiently repeat each number in the sequence

    return repeated_array

In [20]:
import numpy as np
generate_repeated_array(10)

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1])

In [6]:
a = coppock_strategy.generate_signals(all_tickers[200])

In [7]:
a


,close,high,low,signal,position,return,cumulative_return,exit_type,strength
date,,,,,,,,,
2025-01-08,3904.800049,3960.600098,3878.800049,0,0.0,0.0,0.0,none,NaN
2025-01-09,3946.000000,4249.000000,3870.500000,0,0.0,0.0,0.0,none,-1.000000
2025-01-10,3793.800049,3947.500000,3767.050049,0,0.0,0.0,0.0,none,-1.000000
2025-01-13,3726.600098,3750.000000,3529.300049,0,0.0,0.0,0.0,none,-1.000000
2025-01-14,3666.600098,3735.699951,3620.000000,0,0.0,0.0,0.0,none,-1.000000
2025-01-15,3741.750000,3765.000000,3623.399902,0,0.0,0.0,0.0,none,-1.000000
2025-01-16,3821.199951,3848.649902,3657.800049,0,0.0,0.0,0.0,none,-0.939279
2025-01-17,3775.750000,3815.399902,3755.850098,0,0.0,0.0,0.0,none,-0.852946
2025-01-20,3755.250000,3773.300049,3716.050049,0,0.0,0.0,0.0,none,-0.742784


In [14]:
from src.strategies.momentum.disparity_index_strat import DisparityIndexStrategy

In [15]:
di_strat = DisparityIndexStrategy(db_config)

In [16]:
a = di_strat.generate_signals(all_tickers[0])

In [19]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    print(i)
    a = di_strat.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
from src.strategies.momentum.know_sure_thing_strat import KSTStrategy

In [5]:
kst_strat = KSTStrategy(db_config)

In [6]:
a = kst_strat.generate_signals(all_tickers[0])

In [7]:
a

,date,close,kst,signal_line,signal,signal_strength
0,2024-11-26 00:00:00.000000,1785.400024,NaN,NaN,0,NaN
1,2024-11-27 00:00:00.000000,1811.000000,NaN,NaN,0,NaN
2,2024-11-28 00:00:00.000000,1792.900024,NaN,NaN,0,NaN
3,2024-11-29 00:00:00.000000,1797.650024,NaN,NaN,0,NaN
4,2024-12-02 00:00:00.000000,1804.550049,NaN,NaN,0,NaN
5,2024-12-03 00:00:00.000000,1826.849976,NaN,NaN,0,NaN
6,2024-12-04 00:00:00.000000,1860.050049,NaN,NaN,0,NaN
7,2024-12-05 00:00:00.000000,1863.800049,NaN,NaN,0,NaN
8,2024-12-06 00:00:00.000000,1856.500000,NaN,NaN,0,NaN
9,2024-12-09 00:00:00.000000,1869.800049,NaN,NaN,0,NaN


In [8]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    print(i)
    a = kst_strat.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


Insufficient data for analysis


265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507


In [1]:
import sys
import os

# Determine the absolute path to the root directory (assuming the script is one level down)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to sys.path if it’s not already there
if root_path not in sys.path:
    sys.path.insert(0, root_path)

#########################################################
from src.database.config import DatabaseConfig
from src.database.engine import create_db_engine
db_config = DatabaseConfig.default()

#########################################################
import pandas as pd

tickers = pd.read_excel("../data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()


In [2]:
from src.strategies.momentum.mcad_strat import MACDStrategy

In [3]:
mcad_strat = MACDStrategy(db_config)

In [4]:
a = mcad_strat.generate_signals(all_tickers[0])

/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

In [5]:
for i in range(0, len(all_tickers)):
    ticker = all_tickers[i]
    print(i)
    a = mcad_strat.generate_signals(ticker)
    a['ticker'] = ticker
    if i == 0:
        df = a
    else:
        df = pd.concat([df, a])


0


/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

1


/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

2


...

/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

506


/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

507


/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_above = above.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev_below = below.shift(1).fillna(False)
/home/bhvignesh/trading_system/src/strategies/momentum/mcad_strat.py:114: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  positions = signals.replace(0, method='ffill').fill

In [1]:
import sys
import os

# Determine the absolute path to the root directory (assuming the script is one level down)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to sys.path if it’s not already there
if root_path not in sys.path:
    sys.path.insert(0, root_path)

#########################################################
from src.database.config import DatabaseConfig
from src.database.engine import create_db_engine
db_config = DatabaseConfig.default()

#########################################################
import pandas as pd

tickers = pd.read_excel("../data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()


In [3]:
from src.strategies.momentum.relative_strength_index_strat import RSIStrategy

In [5]:
rsi_strat = RSIStrategy(db_config)

In [6]:
a = rsi_strat.generate_signals(all_tickers[0])

In [7]:
a

,signal,signal_strength,rsi
date,,,
2025-02-07 00:00:00.000000,0,0.000000,57.278312
2025-02-06 00:00:00.000000,0,0.000000,60.785211
2025-02-05 00:00:00.000000,0,0.000000,59.468467
2025-02-04 00:00:00.000000,0,0.000000,56.602496
2025-02-03 00:00:00.000000,0,0.000000,46.994659
...,...,...,...
2024-11-18 00:00:00.000000,0,0.000000,67.671297
2024-11-14 00:00:00.000000,0,0.000000,65.891271
2024-11-13 00:00:00.000000,-1,0.200773,63.976808


In [1]:
import sys
import os

# Determine the absolute path to the root directory (assuming the script is one level down)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to sys.path if it’s not already there
if root_path not in sys.path:
    sys.path.insert(0, root_path)

#########################################################
from src.database.config import DatabaseConfig
from src.database.engine import create_db_engine
db_config = DatabaseConfig.default()

#########################################################
import pandas as pd

tickers = pd.read_excel("../data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()

In [2]:
from src.strategies.momentum.relative_vigor_index_strat import RVIStrategy

In [3]:
rvi_strat = RVIStrategy(db_config)

In [4]:
a = rvi_strat.generate_signals(all_tickers[0])

In [5]:
a

,close,rvi,signal_line,signal,signal_strength
date,,,,,
2025-01-10 00:00:00.000000,1657.150024,-0.223669,-0.037278,0,NaN
2025-01-13 00:00:00.000000,1630.099976,-0.257329,-0.117444,0,NaN
2025-01-14 00:00:00.000000,1644.250000,-0.291554,-0.208925,0,NaN
2025-01-15 00:00:00.000000,1642.750000,-0.325453,-0.274481,0,NaN
2025-01-16 00:00:00.000000,1652.750000,-0.356942,-0.308048,0,NaN
2025-01-17 00:00:00.000000,1636.699951,-0.381641,-0.339664,0,NaN
2025-01-20 00:00:00.000000,1651.250000,-0.390609,-0.365538,0,NaN
2025-01-21 00:00:00.000000,1641.750000,-0.380638,-0.380346,0,NaN
2025-01-22 00:00:00.000000,1665.050049,-0.350483,-0.379103,1,NaN
